# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.


### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                        |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [1]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
import requests  # Add this import
from strands.models import BedrockModel

# Create a custom tool 
@tool
def weather(city: str) -> str:
    """
    Fetches weather data directly from OpenWeatherMap API.
    """
    api_key = "3f594ec300393cbc67cffecf93faf528"
    if not api_key:
        raise RuntimeError("OPENWEATHERMAP_API_KEY not set in environment")

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        weather_description = data["weather"][0]["description"].capitalize()
        temperature = data["main"]["temp"]
        feels_like = data["main"]["feels_like"]
        humidity = data["main"]["humidity"]

        return (
            f"Weather in {city.title()}:\n"
            f"- Condition: {weather_description}\n"
            f"- Temperature: {temperature}°C (feels like {feels_like}°C)\n"
            f"- Humidity: {humidity}%"
        )

    except requests.RequestException as e:
        return f"⚠️ Error fetching weather for {city}: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"⚠️ Unexpected response format for {city}: {str(e)}"


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)  # Add print to see the output

Overwriting strands_claude.py


#### Invoking local agent

In [4]:
!python strands_claude.py '{"prompt": "What is the weather today in Delhi"}'


Tool #1: weather
Here's the weather in Delhi today:

- **Condition**: Haze
- **Temperature**: 14.05°C (feels like 13.26°C)
- **Humidity**: 67%

It's a relatively cool day with haziness in the air. You might want to wear a light jacket if you're heading out!Here's the weather in Delhi today:

- **Condition**: Haze
- **Temperature**: 14.05°C (feels like 13.26°C)
- **Humidity**: 67%

It's a relatively cool day with haziness in the air. You might want to wear a light jacket if you're heading out!


## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [5]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
import requests  # Add this import
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
# Create a custom tool 
@tool
def weather(city: str) -> str:
    """
    Fetches weather data directly from OpenWeatherMap API.
    """
    api_key = "3f594ec300393cbc67cffecf93faf528"
    if not api_key:
        raise RuntimeError("OPENWEATHERMAP_API_KEY not set in environment")

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        weather_description = data["weather"][0]["description"].capitalize()
        temperature = data["main"]["temp"]
        feels_like = data["main"]["feels_like"]
        humidity = data["main"]["humidity"]

        return (
            f"Weather in {city.title()}:\n"
            f"- Condition: {weather_description}\n"
            f"- Temperature: {temperature}°C (feels like {feels_like}°C)\n"
            f"- Humidity: {humidity}%"
        )

    except requests.RequestException as e:
        return f"⚠️ Error fetching weather for {city}: {str(e)}"
    except (KeyError, IndexError) as e:
        return f"⚠️ Unexpected response format for {city}: {str(e)}"


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting strands_claude.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [6]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/workspaces/aws-agentCore-bedrock/strands_claude.py, bedrock_agentcore_name=strands_claude
Configuring BedrockAgentCore agent: strands_claude_getting_started


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /workspaces/aws-agentCore-bedrock/Dockerfile
Generated .dockerignore: /workspaces/aws-agentCore-bedrock/.dockerignore
Keeping 'strands_claude_getting_started' as default agent
Bedrock AgentCore configured: /workspaces/aws-agentCore-bedrock/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/workspaces/aws-agentCore-bedrock/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/workspaces/aws-agentCore-bedrock/Dockerfile'), dockerignore_path=PosixPath('/workspaces/aws-agentCore-bedrock/.dockerignore'), runtime='Docker', region='us-east-1', account_id='490688005774', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [7]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started' to account 490688005774 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started
✅ ECR repository available: 490688005774.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started
Getting or creating execution role for agent: strands_claude_getting_started
Using AWS region: us-east-1, account ID: 490688005774
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504


✅ Reusing existing ECR repository: 490688005774.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started


✅ Reusing existing execution role: arn:aws:iam::490688005774:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
✅ Execution role available: arn:aws:iam::490688005774:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_getting_started
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-8556fc4504
Reusing existing CodeBuild execution role: arn:aws:iam::490688005774:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-8556fc4504
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_claude_getting_started/source.zip
Updated CodeBuild project: bedrock-agentcore-strands_claude_getting_started-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.3s
🔄 PROVISIONING started (total: 2s)
✅ PROVISIONING completed in 6.2s
🔄 DOWNLOAD_SOURCE started (t

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [8]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_getting_started


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [9]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now Bhubaneswar?"})
invoke_response

{'ResponseMetadata': {'RequestId': '4fe4bbf6-1177-4dd8-a561-ee3fed52d62b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 21 Jan 2026 15:38:29 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4fe4bbf6-1177-4dd8-a561-ee3fed52d62b',
   'baggage': 'Self=1-6970f2f2-00814e910509f3c232381654,session.id=938b22d5-04d0-4d85-9678-bd16917eb4ec',
   'x-amzn-bedrock-agentcore-runtime-session-id': '938b22d5-04d0-4d85-9678-bd16917eb4ec',
   'x-amzn-trace-id': 'Root=1-6970f2f2-76be26fd773b6a26236d007b;Parent=fd6bc143d300a3ac;Sampled=1;Self=1-6970f2f2-00814e910509f3c232381654'},
  'RetryAttempts': 0},
 'runtimeSessionId': '938b22d5-04d0-4d85-9678-bd16917eb4ec',
 'traceId': 'Root=1-6970f2f2-76be26fd773b6a26236d007b;Parent=fd6bc143d300a3ac;Sampled=1;Self=1-6970f2f2-00814e910509f3c232381654',
 'baggage': 'Self=1-6970f2f2-00814e910509f3c232381654,session.id=938b22d5-04d0-4d85-9678-bd16917eb4ec',
 'contentType': 

### Processing invocation results

We can now process our invocation results to include it in an application

In [10]:
from IPython.display import Markdown, display
import json
response_text = invoke_response['response'][0]
display(Markdown(response_text))

The weather in Bhubaneswar right now is:

- **Condition**: Haze
- **Temperature**: 19.12°C (feels like 18.97°C)
- **Humidity**: 72%

It's a bit hazy today with cool temperatures and moderate humidity. You might want to carry a light jacket if you're going out!

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [11]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather in Pune?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The weather in Pune is currently:
- **Condition**: Clear sky ☀️
- **Temperature**: 23.56°C (feels like 22.83°C)
- **Humidity**: 33%

It's a pleasant day with clear skies and moderate humidity!

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [12]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('490688005774.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started',
 'strands_claude_getting_started-Alwho8Hlhg',
 'bedrock-agentcore-strands_claude_getting_started')

In [13]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!